In [17]:
import shap # 0.46.0
import pandas as pd
import numpy as np
import requests
import time
import os

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import xgboost as xgb # 2.1.1
import matplotlib.pyplot as plt # 3.9.2
import holidays # 0.56
from geopy.geocoders import Nominatim
import pickle

train = pd.read_csv('train_taxi_tims.csv', encoding='utf-8-sig', engine='python')
test = pd.read_csv('test_taxi_tims.csv', encoding='utf-8-sig', engine='python')

train = train[(train['x_axis'] != 0) & (train['y_axis'] != 0)]
test = test[(test['x_axis'] != 0) & (test['y_axis'] != 0)]

train['datetime'] = pd.to_datetime(train['datetime'])
train['minute'] = train['datetime'].dt.minute
train['minute'] = train['minute'] // 10 * 10
train['hour'] = train['datetime'].dt.hour
train['weekday'] = train['datetime'].dt.weekday
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['year'] = train['datetime'].dt.year
train.drop(['datetime'], axis=1, inplace=True)

test['datetime'] = pd.to_datetime(test['datetime'])
test['minute'] = test['datetime'].dt.minute
test['minute'] = test['minute'] // 10 * 10
test['hour'] = test['datetime'].dt.hour
test['weekday'] = test['datetime'].dt.weekday
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['year'] = test['datetime'].dt.year
test.drop(['datetime'], axis=1, inplace=True)


API = "gGryFchORUuq8hXITjVLWQ"
def get_weather(year, month, day, hour, minute):
    url = f"https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm1={time_str}&stn=133&help=1&authKey={API}"

    response = requests.get(url)
    raw_data = [i for i in response.text.split("\n")[-3].split(' ')if i!='']
    WS, TEMP, HUMI, RN = raw_data[3], raw_data[11], raw_data[13], raw_data[15]
    return WS, TEMP, HUMI, RN

In [18]:
if 'weather_data.csv' not in os.listdir():
    data = []
    for month in range(3,12):
        url = f"https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1=2023{str(month).zfill(2)}010000&tm2=2023{str(month+1).zfill(2)}010000&stn=133&help=0&authKey={API}"
        response = requests.get(url)
        raw_data = [[i for i in response.text.split("\n")[j].split(' ')if i!=''] for j in range(4,len(response.text.split("\n")))]
        data.append(raw_data)

    for month in range(1,5):
        url = f"https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1=2024{str(month).zfill(2)}010000&tm2=2024{str(month+1).zfill(2)}010000&stn=133&help=0&authKey={API}"
        response = requests.get(url)
        raw_data = [[i for i in response.text.split("\n")[j].split(' ')if i!=''] for j in range(4,len(response.text.split("\n")))]
        data.append(raw_data)

    total_data = []
    for i in range(len(data)):
        for j in range(len(data[i])-1):
            try:
                new_data = [data[i][j][0], data[i][j][3], data[i][j][11], data[i][j][13], data[i][j][15]]
                total_data.append(new_data)
            except:
                continue

    #make the data into csv
    total_data = pd.DataFrame(total_data, columns=['datetime', 'WS', 'TEMP', 'HUMI', 'RN'])
    total_data.to_csv('weather_data.csv', index=False)

else:
    total_data = pd.read_csv('weather_data.csv', encoding='utf-8-sig', engine='python')

# change ['datetime'] to datetime type with format 'YYYYMMDDHHMM'
total_data['datetime'] = pd.to_datetime(total_data['datetime'], format='%Y%m%d%H%M')
total_data['hour'] = total_data['datetime'].dt.hour
total_data['month'] = total_data['datetime'].dt.month
total_data['day'] = total_data['datetime'].dt.day
total_data['year'] = total_data['datetime'].dt.year
total_data['holiday'] = total_data['datetime'].apply(lambda x: 1 if x in holidays.KR() else 0)
total_data.drop(['datetime'], axis=1, inplace=True)
train = pd.merge(train, total_data, on=['year', 'month', 'day', 'hour'], how='left')
test = pd.merge(test, total_data, on=['year', 'month', 'day', 'hour'], how='left')


In [19]:
cluster_num = 50
if 'kmeans_model.pkl' not in os.listdir():
    # cluster by x_axis and y_axis
    kmeans = KMeans(n_clusters=cluster_num, random_state=5).fit(train[['x_axis', 'y_axis']])
    train['cluster'] = kmeans.predict(train[['x_axis', 'y_axis']])
    test['cluster'] = kmeans.predict(test[['x_axis', 'y_axis']])

    # save the kmeans model
    import pickle
    with open('kmeans_model.pkl', 'wb') as f:
        pickle.dump(kmeans, f)

else:
    with open('kmeans_model.pkl', 'rb') as f:
        kmeans = pickle.load(f)
    train['cluster'] = kmeans.predict(train[['x_axis', 'y_axis']])
    test['cluster'] = kmeans.predict(test[['x_axis', 'y_axis']])

In [20]:
ktv = pd.read_csv('/home/sb/taxi/data/Drinks/dj_ktv.csv', encoding='utf-8-sig', engine='python')
karaoke = pd.read_csv('/home/sb/taxi/data/Drinks/dj_karaoke.csv', encoding='utf-8-sig', engine='python')
hospital = pd.read_csv('/home/sb/taxi/data/Hospitals/dj_hospital.csv', encoding='utf-8-sig', engine='python')
small_hospital = pd.read_csv('/home/sb/taxi/data/Hospitals/dj_small_hospital.csv', encoding='utf-8-sig', engine='python')
camping = pd.read_csv('/home/sb/taxi/data/Hotels/dj_camping.csv', encoding='utf-8-sig', engine='python')
country_hotel = pd.read_csv('/home/sb/taxi/data/Hotels/dj_country_hotel.csv', encoding='utf-8-sig', engine='python')
foreign_hotel = pd.read_csv('/home/sb/taxi/data/Hotels/dj_foreign_hotel.csv', encoding='utf-8-sig', engine='python')
hotel = pd.read_csv('/home/sb/taxi/data/Hotels/dj_hotel.csv', encoding='utf-8-sig', engine='python')
tour_hotel = pd.read_csv('/home/sb/taxi/data/Hotels/dj_tour_hotel.csv', encoding='utf-8-sig', engine='python')

In [21]:
# rename 경도 to x_axis, 위도 to y_axis
ktv.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
karaoke.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
hospital.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
small_hospital.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
camping.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
country_hotel.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
foreign_hotel.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
hotel.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)
tour_hotel.rename(columns={'경도':'x_axis', '위도':'y_axis'}, inplace=True)

In [22]:
ktv['cluster'] = kmeans.predict(ktv[['x_axis', 'y_axis']])
karaoke['cluster'] = kmeans.predict(karaoke[['x_axis', 'y_axis']])
hospital['cluster'] = kmeans.predict(hospital[['x_axis', 'y_axis']])
small_hospital['cluster'] = kmeans.predict(small_hospital[['x_axis', 'y_axis']])
camping['cluster'] = kmeans.predict(camping[['x_axis', 'y_axis']])
country_hotel['cluster'] = kmeans.predict(country_hotel[['x_axis', 'y_axis']])
foreign_hotel['cluster'] = kmeans.predict(foreign_hotel[['x_axis', 'y_axis']])
hotel['cluster'] = kmeans.predict(hotel[['x_axis', 'y_axis']])
tour_hotel['cluster'] = kmeans.predict(tour_hotel[['x_axis', 'y_axis']])

In [23]:
hospital['업태구분명'] = pd.Categorical(hospital['업태구분명']).codes
number_of_hospital_type = len(hospital['업태구분명'].unique())

small_hospital['업태구분명'] = pd.Categorical(small_hospital['업태구분명']).codes
number_of_small_hospital_type = len(small_hospital['업태구분명'].unique())

new_columns = ['sum_drinks', 'sum_hospitals', 'sum_hotels', 'sum_drinks_area', 
               'sum_hospital_rooms'] + [f'sum_hospital_type_{i}' for i in range(number_of_hospital_type)] + [f'sum_small_hospital_type_{i}' for i in range(number_of_small_hospital_type)]
train[new_columns] = 0

drink_dfs = [ktv, karaoke]
hospital_dfs = [hospital, small_hospital]
hotel_dfs = [hotel]


def sum_for_cluster(df, cluster, column=None):
    mask = df['cluster'] == cluster
    return len(df[mask]) if column is None else df.loc[mask, column].sum()

In [24]:
for i in range(cluster_num):
    cluster_mask = train['cluster'] == i
    test_cluster_mask = test['cluster'] == i
    
    # Sum drinks and drink areas
    train.loc[cluster_mask, 'sum_drinks'] = sum(sum_for_cluster(df, i) for df in drink_dfs)
    train.loc[cluster_mask, 'sum_drinks_area'] = sum(sum_for_cluster(df, i, '시설총규모') for df in drink_dfs)
    
    # Sum hospitals and hospital rooms
    train.loc[cluster_mask, 'sum_hospitals'] = sum(sum_for_cluster(df, i) for df in hospital_dfs)
    train.loc[cluster_mask, 'sum_hospital_rooms'] = sum(sum_for_cluster(df, i, '병상수') for df in hospital_dfs)
    
    # Sum hospital types
    for j in range(number_of_hospital_type):
        train.loc[cluster_mask, f'sum_hospital_type_{j}'] = sum(
            sum_for_cluster(df[df['업태구분명'] == j], i) for df in hospital_dfs
        )
    
    for k in range(number_of_small_hospital_type):
        train.loc[cluster_mask, f'sum_small_hospital_type_{k}'] = sum(
            sum_for_cluster(df[df['업태구분명'] == k], i) for df in hospital_dfs
        )
    
    # Sum hotels
    train.loc[cluster_mask, 'sum_hotels'] = sum(sum_for_cluster(df, i) for df in hotel_dfs)

    # Test data
    test.loc[test_cluster_mask, 'sum_drinks'] = sum(sum_for_cluster(df, i) for df in drink_dfs)
    test.loc[test_cluster_mask, 'sum_drinks_area'] = sum(sum_for_cluster(df, i, '시설총규모') for df in drink_dfs)

    test.loc[test_cluster_mask, 'sum_hospitals'] = sum(sum_for_cluster(df, i) for df in hospital_dfs)
    test.loc[test_cluster_mask, 'sum_hospital_rooms'] = sum(sum_for_cluster(df, i, '병상수') for df in hospital_dfs)

    for j in range(number_of_hospital_type):
        test.loc[test_cluster_mask, f'sum_hospital_type_{j}'] = sum(
            sum_for_cluster(df[df['업태구분명'] == j], i) for df in hospital_dfs
        )

    for k in range(number_of_small_hospital_type):
        test.loc[test_cluster_mask, f'sum_small_hospital_type_{k}'] = sum(
            sum_for_cluster(df[df['업태구분명'] == k], i) for df in hospital_dfs
        )

    test.loc[test_cluster_mask, 'sum_hotels'] = sum(sum_for_cluster(df, i) for df in hotel_dfs)

/tmp/ipykernel_265445/1450656950.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2116.95' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train.loc[cluster_mask, 'sum_drinks_area'] = sum(sum_for_cluster(df, i, '시설총규모') for df in drink_dfs)


In [25]:
# 동시간대에 cluster 별로 trip의 수를 count
train['count'] = 1
train_count = train.groupby(['cluster', 'minute', 'hour', 'day', 'month', 'year']).count().reset_index()
train_count = train_count[['cluster', 'minute', 'hour', 'day', 'month', 'year','count']]

train = pd.merge(train, train_count, on=['cluster', 'minute', 'hour', 'day', 'month', 'year'], how='left')
train['count'] = train['count_y']
train.drop(['count_x', 'count_y'], axis=1, inplace=True)

test['count'] = 1
test_count = test.groupby(['cluster', 'minute', 'hour', 'day', 'month', 'year']).count().reset_index()
test_count = test_count[['cluster', 'minute', 'hour', 'day', 'month', 'year','count']]

test = pd.merge(test, test_count, on=['cluster', 'minute', 'hour', 'day', 'month', 'year'], how='left')
test['count'] = test['count_y']
test.drop(['count_x', 'count_y'], axis=1, inplace=True)

In [26]:
#save the data
train.to_csv('train_taxi_50.csv', index=False)
test.to_csv('test_taxi_50.csv', index=False)

In [31]:
# xgb regression
model = xgb.XGBRegressor(n_estimators=1000, max_depth=10, learning_rate=0.05, random_state=5)
train_columns = ['hour', 'weekday', 'month', 'day', 'WS', 'TEMP', 'HUMI', 'RN', 'cluster', 'holiday'] + new_columns

model.fit(train[train_columns], train['count'])
#save the model

with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [32]:
# test accuracy
pred = model.predict(test[train_columns])
print(np.mean(np.abs(pred - test['count'])))

0.7694412663529593


In [9]:
# shap
explainer = shap.Explainer(model, test[train_columns])
#save the explainer
with open('explainer.pkl', 'wb') as f:
    pickle.dump(explainer, f)

In [13]:
import folium
import pandas as pd

# cluster centers
data = {
    'name': ['cluster_' + str(i) for i in range(cluster_num)],
    'latitude': [i[1] for i in kmeans.cluster_centers_],
    'longitude': [i[0] for i in kmeans.cluster_centers_]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Calculate the center of the map
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

# Create a map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers to the map
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        tooltip=row['name']
    ).add_to(m)

# Save the map
m.save("map_with_markers.html")

print("Map has been saved as 'map_with_markers.html'")

Map has been saved as 'map_with_markers.html'


In [14]:
minute = None
# load the kmeans model
with open('kmeans_model.pkl', 'rb') as f:
    kmeans = pickle.load(f)

# load the xgb model
with open('xgb_model.pkl', 'rb') as f:
    model = pickle.load(f)

# load the explainer
with open('explainer.pkl', 'rb') as f:
    explainer = pickle.load(f)

cluster_centers = kmeans.cluster_centers_

In [15]:
print(cluster_centers)

[[127.32078586  36.37711171]
 [127.40598292  36.32439321]
 [127.38271699  36.36960215]
 [127.45697618  36.30374648]
 [127.42511807  36.44750205]
 [127.33876665  36.30440491]
 [127.4309554   36.35705376]
 [127.34283361  36.35455921]
 [127.37912166  36.31428959]
 [127.39446659  36.41901964]
 [127.42962293  36.32812206]
 [127.39042386  36.34477034]
 [127.25611147  36.50098314]
 [127.37319833  36.35085593]
 [127.44949836  36.34035747]]


In [23]:
while True:
    time.sleep(1)
    temp_time = time.strftime('%Y%m%d%H%M', time.localtime(time.time()))
    if minute == int(temp_time[10:12]):
        continue
    else:
        year, month, day, hour, minute = int(temp_time[:4]), int(temp_time[4:6]), int(temp_time[6:8]), int(temp_time[8:10]), int(temp_time[10:12])
        weekday = pd.to_datetime(f'{year}-{month}-{day}').weekday()
        minute = minute // 10 * 10
        WS, TEMP, HUMI, RN = get_weather(year, month, day, hour, minute)
        holiday = 1 if f'{year}{str(month).zfill(2)}{day}' in holidays.KR() else 0
        for i in range(cluster_num):
            x_axis, y_axis = cluster_centers[i]
            # kmeans.predict([[x_axis, y_axis]])
            new_data = pd.DataFrame([[x_axis, y_axis, minute, hour, day, weekday, month, year, WS, TEMP, HUMI, RN, i, holiday]], columns=['x_axis', 'y_axis', 'minute', 'hour', 'day', 'weekday', 'month', 'year', 'WS', 'TEMP', 'HUMI', 'RN', 'cluster', 'holiday'])
            new_data['WS'] = new_data['WS'].astype(float)
            new_data['TEMP'] = new_data['TEMP'].astype(float)
            new_data['HUMI'] = new_data['HUMI'].astype(float)
            new_data['RN'] = new_data['RN'].astype(float)
            pred = model.predict(new_data[train_columns])
            shap_values = explainer(new_data[train_columns])
            # shap.plots.waterfall(shap_values[0])
            print(f'cluster_{i} : {pred[0]}')

        break


cluster_0 : 1.5999970436096191
cluster_1 : 3.3072245121002197
cluster_2 : 2.6796555519104004
cluster_3 : 2.268153429031372
cluster_4 : 0.9975440502166748
cluster_5 : 1.8796712160110474
cluster_6 : 4.565709114074707
cluster_7 : 3.08294415473938
cluster_8 : 2.359192371368408
cluster_9 : 1.9838122129440308
cluster_10 : 6.028375148773193
cluster_11 : 4.285617351531982
cluster_12 : 0.6500904560089111
cluster_13 : 3.5307488441467285
cluster_14 : 1.8716413974761963
